# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

10-04-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-10-05 04:23:06,33.93911,67.709953,199690,7803,NaN,NaN,Afghanistan,512.968174,3.907557
1,NaN,NaN,NaN,Albania,2022-10-05 04:23:06,41.15330,20.168300,332337,3589,NaN,NaN,Albania,11548.300785,1.079928
2,NaN,NaN,NaN,Algeria,2022-10-05 04:23:06,28.03390,1.659600,270690,6879,NaN,NaN,Algeria,617.294325,2.541283
3,NaN,NaN,NaN,Andorra,2022-10-05 04:23:06,42.50630,1.521800,46227,155,NaN,NaN,Andorra,59829.159387,0.335302
4,NaN,NaN,NaN,Angola,2022-10-05 04:23:06,-11.20270,17.873900,103131,1917,NaN,NaN,Angola,313.789810,1.858801


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22,9/30/22,10/1/22,10/2/22,10/3/22,10/4/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,198416,198543,198750,198876,199067,199188,199310,199386,199545,199690
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,331976,331987,332066,332129,332173,332221,332263,332285,332290,332337
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270641,270649,270654,270662,270668,270673,270676,270679,270682,270690


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22,9/30/22,10/1/22,10/2/22,10/3/22,10/4/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7796,7798,7798,7799,7799,7800,7800,7801,7803,7803
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3589,3589,3589,3589,3589,3589,3589,3589,3589,3589
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6879,6879,6879,6879,6879,6879,6879,6879,6879,6879


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22,9/30/22,10/1/22,10/2/22,10/3/22,10/4/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22,9/30/22,10/1/22,10/2/22,10/3/22,10/4/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18359,18359,18359,18359,18396,18396,18396,18396,18396,18396
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,65544,65544,65544,65544,65653,65653,65653,65653,65653,65653
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6881,6881,6881,6881,6896,6896,6896,6896,6896,6896


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22,9/30/22,10/1/22,10/2/22,10/3/22,10/4/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,227,227,227,227,227,227,227,227,227,227
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,711,711,711,711,712,712,712,712,712,712
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,102,102,102,102,103,103,103,103,103,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2022-10-05 04:23:06,32.539527,-86.644082,18396,227,NaN,NaN,"Autauga, Alabama, US",32927.025721,1.233964
715,1075.0,Lamar,Alabama,US,2022-10-05 04:23:06,33.779950,-88.096680,4381,69,NaN,NaN,"Lamar, Alabama, US",31734.878667,1.574983
716,1077.0,Lauderdale,Alabama,US,2022-10-05 04:23:06,34.901719,-87.656247,28206,408,NaN,NaN,"Lauderdale, Alabama, US",30417.668690,1.446501


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,96481081,1060408,0.0
India,44601892,528733,0.0
France,35790213,156312,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,96481081,1060408,0.0,95420673.0,2022-10-05 04:23:06,37.934269,-91.444083
India,44601892,528733,0.0,44073159.0,2022-10-05 04:23:06,23.088275,81.806127
France,35790213,156312,0.0,35633901.0,2022-10-05 04:23:06,6.430808,-34.730285
Brazil,34684529,686371,0.0,33998158.0,2022-10-05 04:23:06,-12.669522,-48.480493
Germany,33519761,150192,0.0,33369569.0,2022-10-05 04:23:06,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11270213,96071,0.0
Texas,8002447,90940,0.0
Florida,7150188,81464,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11270213,96071,0.0,11174142.0,2022-10-05 04:23:06,37.843962,-120.728594
Texas,8002447,90940,0.0,7911507.0,2022-10-05 04:23:06,31.660643,-98.653069
Florida,7150188,81464,0.0,7068724.0,2022-10-05 04:23:06,28.940755,-82.700744
New York,6137273,71899,0.0,6065374.0,2022-10-05 04:23:06,42.544151,-75.474183
Illinois,3767439,39732,0.0,3727707.0,2022-10-05 04:23:06,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3459997,33709,0.0
Florida,Miami-Dade,1458095,11648,0.0
Arizona,Maricopa,1424974,17940,0.0
Illinois,Cook,1409288,14812,0.0
Texas,Harris,1215364,11311,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3459997,33709,0.0,3426288.0,2022-10-05 04:23:06,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1458095,11648,0.0,1446447.0,2022-10-05 04:23:06,25.611236,-80.551706,12086.0
Arizona,Maricopa,1424974,17940,0.0,1407034.0,2022-10-05 04:23:06,33.348359,-112.491815,4013.0
Illinois,Cook,1409288,14812,0.0,1394476.0,2022-10-05 04:23:06,41.841448,-87.816588,17031.0
Texas,Harris,1215364,11311,0.0,1204053.0,2022-10-05 04:23:06,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,199386,332285,270679,46227,103131,11,9098,9711355,442875,10241739,...,985422,244218,11950,544874,11480518,702768,535,11935,333549,257517
2022-10-03,199545,332290,270682,46227,103131,11,9098,9711355,443785,10243195,...,986446,244245,11974,544884,11481314,702768,535,11935,333555,257538
2022-10-04,199690,332337,270690,46227,103131,11,9098,9711355,443785,10244727,...,986446,244257,11974,544884,11482334,702768,535,11935,333555,257568


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,7801,3589,6879,155,1917,0,146,129937,8683,15221,...,7485,1637,14,5816,43149,5707,0,2157,4017,5602
2022-10-03,7803,3589,6879,155,1917,0,146,129937,8691,15228,...,7495,1637,14,5816,43149,5707,0,2157,4017,5603
2022-10-04,7803,3589,6879,155,1917,0,146,129937,8691,15234,...,7495,1637,14,5816,43151,5707,0,2157,4017,5603


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-10-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-10-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22,9/30/22,10/1/22,10/2/22,10/3/22,10/4/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18359,18359,18359,18359,18396,18396,18396,18396,18396,18396
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4375,4375,4375,4375,4381,4381,4381,4381,4381,4381
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,28147,28147,28147,28147,28206,28206,28206,28206,28206,28206
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,8843,8843,8843,8843,8862,8862,8862,8862,8862,8862
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,44084,44084,44084,44084,44149,44149,44149,44149,44149,44149


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-10-02,1522135,298178,8247,2271560,952004,11266623,1654798,898073,308713,49,...,2338638,7996241,1038416,142443,23166,2087360,1816183,600558,1866634,176728
2022-10-03,1522135,298178,8247,2271560,952123,11269064,1654798,898338,309189,49,...,2339630,8001850,1038416,142443,23177,2091185,1818645,601246,1868896,176728
2022-10-04,1522135,298178,8247,2271560,952469,11270213,1656683,899205,309265,49,...,2341239,8002447,1038416,142443,23187,2092255,1818714,601520,1870010,177165


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-10-02       18396   65653    6896  7526  16912    2802   6051   38211   
2022-10-03       18396   65653    6896  7526  16912    2802   6051   38211   
2022-10-04       18396   65653    6896  7526  16912    2802   6051   38211   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-10-02         9985     6008  ...    7436   2141     9251     2186   
2022-10-03         9985     6008  ...    7436   2141     9251     2186   
2022-10-04         9985     6008  ...    7446   2149     9281     2205   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-10-02          12143  11699  6145          0     2662   1814  
2022-10-03          12143  11699  6145          0     2662   1814  
2022-10-04          12147  11733  6155          0     2665   1814  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-10-02,20422,1364,34,31370,12181,95965,13316,11365,3111,0,...,27816,90913,5021,719,123,21849,14339,7416,15263,1894
2022-10-03,20422,1364,34,31370,12181,95990,13316,11365,3111,0,...,27818,90940,5021,719,123,21884,14347,7426,15263,1894
2022-10-04,20422,1364,34,31370,12213,96071,13319,11385,3112,0,...,27875,90940,5021,719,123,21901,14344,7436,15272,1899


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-10-02         227     712     103  107    258      54    129     656   
2022-10-03         227     712     103  107    258      54    129     656   
2022-10-04         227     712     103  107    258      54    129     656   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-10-02          169       87  ...     150     43       82       28   
2022-10-03          169       87  ...     150     43       82       28   
2022-10-04          169       87  ...     150     43       82       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-10-02            130    16    41          0       47     21  
2022-10-03            130    16    41          0       47     21  
2022-10-04            130    16    42          0       47     21  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,0.000381,0.000066,0.000011,0.0,0.0,0.0,0.0,0.000302,0.000000,0.000108,...,0.000000,0.000000,0.000000,0.000026,0.000043,0.0,0.0,0.0,0.000000,0.000202
2022-10-03,0.000797,0.000015,0.000011,0.0,0.0,0.0,0.0,0.000000,0.002055,0.000142,...,0.001039,0.000111,0.002008,0.000018,0.000069,0.0,0.0,0.0,0.000018,0.000082
2022-10-04,0.000727,0.000141,0.000030,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000150,...,0.000000,0.000049,0.000000,0.000000,0.000089,0.0,0.0,0.0,0.000000,0.000116


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,0.000128,0.0,0.0,0.0,0.0,NaN,0.0,0.000308,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,0.0,0.000000
2022-10-03,0.000256,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.000921,0.000460,...,0.001336,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,0.0,0.000179
2022-10-04,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.000000,0.000394,...,0.000000,0.0,0.0,0.0,0.000046,0.0,NaN,0.0,0.0,0.000000


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-10-02,0.0,0.0,0.0,0.0,0.000118,0.000000,0.000000,0.000417,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-10-03,0.0,0.0,0.0,0.0,0.000125,0.000217,0.000000,0.000295,0.001542,0.0,...,0.000424,0.000701,0.0,0.0,0.000475,0.001832,0.001356,0.001146,0.001212,0.000000
2022-10-04,0.0,0.0,0.0,0.0,0.000363,0.000102,0.001139,0.000965,0.000246,0.0,...,0.000688,0.000075,0.0,0.0,0.000432,0.000512,0.000038,0.000456,0.000596,0.002473


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-10-02         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-10-03         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-10-04         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                                \
Admin2         Chambers Cherokee  ...      Park    Platte  Sheridan  Sublette   
2022-10-02          0.0      0.0  ...  0.000000  0.000000  0.000000  0.000000   
2022-10-03          0.0      0.0  ...  0.000000  0.000000  0.000000  0.000000   
2022-10-04          0.0      0.0  ...  0.001345  0.003737  0.003243  0.008692   

Province_State                                                             
Admin2         Sweetwater     Teton     Uinta Unassigned  Washakie Weston  
2022-10-02       0.000000  0.000000  0.000000        NaN  0.000000    0.0  
2022-10-03       0.000000  0.000000  0.000000        NaN  0.000000    0.0  
2022-10-04       0.000329  0.002906  0.001627        NaN  0.001127    0.0  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-10-02,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,NaN,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000
2022-10-03,0.0,0.0,0.0,0.0,0.000000,0.000260,0.000000,0.00000,0.000000,NaN,...,0.000072,0.000297,0.0,0.0,0.0,0.001602,0.000558,0.001348,0.00000,0.00000
2022-10-04,0.0,0.0,0.0,0.0,0.002627,0.000844,0.000225,0.00176,0.000321,NaN,...,0.002049,0.000000,0.0,0.0,0.0,0.000777,-0.000209,0.001347,0.00059,0.00264


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-10-02         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-10-03         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-10-04         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-10-02          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-10-03          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-10-04          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                       
Admin2         Sweetwater Teton    Uinta Unassigned Washakie Weston  
2022-10-02            0.0   0.0  0.00000        NaN      0.0    0.0  
2022-10-03            0.0   0.0  0.00000        NaN      0.0    0.0  
2022-10-04            0.0   0.0  0.02439        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,0.000527,0.000102,0.000014,0.000054,0.0,0.0,0.000132,0.000153,0.000026,0.000284,...,0.000010,0.000017,1.000000e-07,0.000074,0.000066,0.0,0.0,2.000000e-07,0.000022,0.000135
2022-10-03,0.000662,0.000058,0.000012,0.000027,0.0,0.0,0.000066,0.000077,0.001040,0.000213,...,0.000525,0.000064,1.004200e-03,0.000046,0.000068,0.0,0.0,1.000000e-07,0.000020,0.000108
2022-10-04,0.000694,0.000100,0.000021,0.000013,0.0,0.0,0.000033,0.000038,0.000520,0.000181,...,0.000262,0.000056,5.021000e-04,0.000023,0.000078,0.0,0.0,1.000000e-07,0.000010,0.000112


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,0.000086,1.000000e-07,0.0,0.0,0.0,NaN,0.000054,0.000155,0.000004,0.001266,...,0.000013,0.0,0.0,0.000086,0.000008,0.0,NaN,5.000000e-07,0.0,0.000017
2022-10-03,0.000171,0.000000e+00,0.0,0.0,0.0,NaN,0.000027,0.000078,0.000463,0.000863,...,0.000674,0.0,0.0,0.000043,0.000004,0.0,NaN,2.000000e-07,0.0,0.000098
2022-10-04,0.000086,0.000000e+00,0.0,0.0,0.0,NaN,0.000013,0.000039,0.000231,0.000629,...,0.000337,0.0,0.0,0.000022,0.000025,0.0,NaN,1.000000e-07,0.0,0.000049


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-10-02,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-10-03,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-10-04,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-10-02,0.000176,0.000065,0.000024,0.000047,0.000223,0.000137,0.000136,0.000647,0.000193,0.0,...,0.000102,0.000150,0.000114,0.000134,0.000065,0.000084,0.000239,0.000161,0.000149,0.000042
2022-10-03,0.000088,0.000033,0.000012,0.000024,0.000174,0.000177,0.000068,0.000471,0.000868,0.0,...,0.000263,0.000426,0.000057,0.000067,0.000270,0.000959,0.000797,0.000653,0.000680,0.000021
2022-10-04,0.000044,0.000016,0.000006,0.000012,0.000269,0.000139,0.000603,0.000718,0.000557,0.0,...,0.000475,0.000250,0.000029,0.000034,0.000351,0.000735,0.000418,0.000554,0.000638,0.001247


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-10-02      0.000128  0.000105  0.000137  0.000185  0.000139  0.000113   
2022-10-03      0.000064  0.000053  0.000068  0.000092  0.000070  0.000057   
2022-10-04      0.000032  0.000026  0.000034  0.000046  0.000035  0.000028   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-10-02      0.000115  0.001208  0.000070  0.000179  ...  0.000043   
2022-10-03      0.000058  0.000604  0.000035  0.000090  ...  0.000021   
2022-10-04      0.000029  0.000302  0.000017  0.000045  ...  0.000683   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-10-02      0.000029  0.000051  0.000044   0.000009  0.000069  0.000049   
2022-10-03      0.000015  0.000026  0.000022   0.000005  0.000035  0.000024   
2022-10-04      0.001876  0.001634  0.004357   0.000167  0.001470  0.000826   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-10-02      -0.503937  0.000059  0.000009  
2022-10-03      -0.503937  0.000030  0.000004  
2022-10-04      -0.503937  0.000578  0.000002  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-10-02,0.000084,0.0,0.0,0.000044,0.000695,0.000040,0.000160,0.000036,0.000301,NaN,...,0.000209,0.000083,0.000126,1.400000e-06,0.0,0.000108,0.000305,0.000214,0.000089,0.000025
2022-10-03,0.000042,0.0,0.0,0.000022,0.000347,0.000150,0.000080,0.000018,0.000151,NaN,...,0.000140,0.000190,0.000063,7.000000e-07,0.0,0.000855,0.000432,0.000781,0.000044,0.000013
2022-10-04,0.000021,0.0,0.0,0.000011,0.001487,0.000497,0.000152,0.000889,0.000236,NaN,...,0.001095,0.000095,0.000031,3.000000e-07,0.0,0.000816,0.000111,0.001064,0.000317,0.001326


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin   Barbour Bibb    Blount Bullock Butler   
2022-10-02         0.0  0.000092  0.000618  0.0  0.000004     0.0    0.0   
2022-10-03         0.0  0.000046  0.000309  0.0  0.000002     0.0    0.0   
2022-10-04         0.0  0.000023  0.000154  0.0  0.000001     0.0    0.0   

Province_State                               ...       Wyoming         \
Admin2           Calhoun  Chambers Cherokee  ...          Park Platte   
2022-10-02      0.000099  0.000375      0.0  ...  8.000000e-07    0.0   
2022-10-03      0.000050  0.000188      0.0  ...  4.000000e-07    0.0   
2022-10-04      0.000025  0.000094      0.0  ...  2.000000e-07    0.0   

Province_State                                                          \
Admin2         Sheridan Sublette Sweetwater Teton     Uinta Unassigned   
2022-10-02          0.0      0.0        0.0   0.0  0.000000       -1.0   
2022-10-03          0.0      0.0        0.0   0.0  0.000000       -1.0   
2022-10-04          0.0      0.0        0.0   0.0  0.012195       -1.0   

Province_State                  
Admin2         Washakie Weston  
2022-10-02          0.0    0.0  
2022-10-03          0.0    0.0  
2022-10-04          0.0    0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 